## Einleitung

Dieses Notebook demonstriert die grundlegende Datenexploration für das PRECISION-Projekt.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting-Einstellungen
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# PRECISION Farbpalette
PRECISION_COLORS = {
    'primary': '#005a8c',
    'secondary': '#007cb0',
    'accent': '#00a0d2',
    'success': '#4a9b5e',
    'warning': '#e8a027',
    'danger': '#c9302c'
}

print('Setup erfolgreich!')

## 1. Daten laden

In [ ]:
# Seed für Reproduzierbarkeit
np.random.seed(42)

# Synthetische Patientendaten erstellen
n_patients = 200

data = {
    'patient_id': [f'PAT_{i:04d}' for i in range(1, n_patients + 1)],
    'age': np.random.normal(55, 15, n_patients).astype(int).clip(18, 90),
    'gender': np.random.choice(['M', 'F'], n_patients, p=[0.52, 0.48]),
    'bmi': np.random.normal(26, 5, n_patients).round(1).clip(15, 45),
    'blood_pressure_sys': np.random.normal(130, 20, n_patients).astype(int).clip(90, 200),
    'blood_pressure_dia': np.random.normal(80, 12, n_patients).astype(int).clip(50, 120),
    'cholesterol': np.random.normal(200, 40, n_patients).round(0).clip(100, 350),
    'glucose': np.random.normal(100, 25, n_patients).round(0).clip(60, 250),
    'treatment_group': np.random.choice(['A', 'B', 'Placebo'], n_patients, p=[0.4, 0.4, 0.2]),
    'response': np.random.choice([0, 1], n_patients, p=[0.35, 0.65])
}

df = pd.DataFrame(data)
print(f'Datensatz erstellt: {len(df)} Patienten')

## 2. Datenübersicht

In [ ]:
# Erste Zeilen anzeigen
df.head(10)

In [ ]:
# Statistische Zusammenfassung
df.describe()

In [ ]:
# Datentypen und fehlende Werte
print('Datentypen:')
print(df.dtypes)
print('\nFehlende Werte:')
print(df.isnull().sum())

## 3. Visualisierungen

In [ ]:
#| label: fig-age-distribution
#| fig-cap: "Altersverteilung nach Geschlecht"

fig, ax = plt.subplots(figsize=(10, 6))

for gender, color in [('M', PRECISION_COLORS['primary']), ('F', PRECISION_COLORS['accent'])]:
    subset = df[df['gender'] == gender]
    ax.hist(subset['age'], bins=20, alpha=0.6, label=f'{gender} (n={len(subset)})', color=color)

ax.set_xlabel('Alter (Jahre)')
ax.set_ylabel('Anzahl Patienten')
ax.set_title('Altersverteilung nach Geschlecht')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
#| label: fig-bmi-bp
#| fig-cap: "BMI vs. Systolischer Blutdruck"

fig, ax = plt.subplots(figsize=(10, 6))

colors = df['response'].map({0: PRECISION_COLORS['danger'], 1: PRECISION_COLORS['success']})
scatter = ax.scatter(df['bmi'], df['blood_pressure_sys'], c=colors, alpha=0.6, s=50)

ax.set_xlabel('BMI (kg/m²)')
ax.set_ylabel('Systolischer Blutdruck (mmHg)')
ax.set_title('BMI vs. Systolischer Blutdruck')

from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=PRECISION_COLORS['success'], alpha=0.6, label='Responder'),
    Patch(facecolor=PRECISION_COLORS['danger'], alpha=0.6, label='Non-Responder')
]
ax.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

In [ ]:
#| label: fig-treatment-comparison
#| fig-cap: "Behandlungsgruppen-Vergleich"

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot: Cholesterin nach Behandlungsgruppe
groups = df.groupby('treatment_group')['cholesterol'].apply(list)
bp = axes[0].boxplot([groups['A'], groups['B'], groups['Placebo']], 
                      labels=['Gruppe A', 'Gruppe B', 'Placebo'],
                      patch_artist=True)
colors_box = [PRECISION_COLORS['primary'], PRECISION_COLORS['accent'], PRECISION_COLORS['secondary']]
for patch, color in zip(bp['boxes'], colors_box):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[0].set_ylabel('Cholesterin (mg/dL)')
axes[0].set_title('Cholesterin nach Behandlungsgruppe')

# Barplot: Response-Rate nach Behandlungsgruppe
response_rate = df.groupby('treatment_group')['response'].mean() * 100
bars = axes[1].bar(response_rate.index, response_rate.values, 
                   color=[PRECISION_COLORS['primary'], PRECISION_COLORS['accent'], PRECISION_COLORS['secondary']],
                   alpha=0.8)
axes[1].set_ylabel('Response-Rate (%)')
axes[1].set_title('Response-Rate nach Behandlungsgruppe')
axes[1].set_ylim(0, 100)

for bar, val in zip(bars, response_rate.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                 f'{val:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Zusammenfassung

### Wichtige Erkenntnisse:

1. **Datensatz-Größe:** 200 Patienten
2. **Altersverteilung:** Normalverteilt um ~55 Jahre
3. **Geschlechterverteilung:** Ausgeglichen (52% M, 48% F)
4. **Response-Rate:** ~65% über alle Gruppen

In [ ]:
# Zusammenfassende Statistiken
summary = {
    'Gesamtanzahl Patienten': len(df),
    'Durchschnittsalter': f"{df['age'].mean():.1f} Jahre",
    'Anteil männlich': f"{(df['gender'] == 'M').mean() * 100:.1f}%",
    'Durchschnittlicher BMI': f"{df['bmi'].mean():.1f} kg/m²",
    'Response-Rate gesamt': f"{df['response'].mean() * 100:.1f}%"
}

for key, value in summary.items():
    print(f'{key}: {value}')